## 离线数据缓存之离线召回集

这里主要是利用我们前面训练的ALS模型进行协同过滤召回，但是注意，我们ALS模型召回的是用户最感兴趣的类别，而我们需要的是用户可能感兴趣的广告的集合，因此我们还需要根据召回的类别匹配出对应的广告。

所以这里我们除了需要我们训练的ALS模型以外，还需要有一个广告和类别的对应关系。


In [2]:
# spark配置信息
from pyspark import SparkConf
from pyspark.sql import SparkSession

SPARK_APP_NAME = "recallAdSets"
SPARK_URL = "yarn"

conf = SparkConf()    # 创建spark config对象
config = (
	("spark.app.name", SPARK_APP_NAME),    # 设置启动的spark的app名称，没有提供，将随机产生一个名称
	("spark.executor.memory", "2g"),    # 设置该app启动时占用的内存用量，默认1g
	("spark.master", SPARK_URL),    # spark master的地址
    ("spark.executor.cores", "2"),   # 设置spark executor使用的CPU核心数
    ("spark.executor.instances", 1)    # 设置spark executor数量，yarn时起作用
)
# 查看更详细配置及说明：https://spark.apache.org/docs/latest/configuration.html
# 
conf.setAll(config)

# 利用config对象，创建spark session
spark = SparkSession.builder.config(conf=conf).getOrCreate()

In [3]:
#### 获取广告和类别的对应关系
# 从HDFS中加载广告基本信息数据，返回spark dafaframe对象
df = spark.read.csv("hdfs://hadoop-master:9000/workspace/3.rs_project/project1/dataset/ad_feature.csv", header=True)

# 注意：由于本数据集中存在NULL字样的数据，无法直接设置schema，只能先将NULL类型的数据处理掉，然后进行类型转换

from pyspark.sql.types import StructType, StructField, IntegerType, FloatType

# 替换掉NULL字符串，替换掉
df = df.replace("NULL", "-1")

# 更改df表结构：更改列类型和列名称
ad_feature_df = df.\
    withColumn("adgroup_id", df.adgroup_id.cast(IntegerType())).withColumnRenamed("adgroup_id", "adgroupId").\
    withColumn("cate_id", df.cate_id.cast(IntegerType())).withColumnRenamed("cate_id", "cateId").\
    withColumn("campaign_id", df.campaign_id.cast(IntegerType())).withColumnRenamed("campaign_id", "campaignId").\
    withColumn("customer", df.customer.cast(IntegerType())).withColumnRenamed("customer", "customerId").\
    withColumn("brand", df.brand.cast(IntegerType())).withColumnRenamed("brand", "brandId").\
    withColumn("price", df.price.cast(FloatType()))

# 这里我们只需要adgroupId、和cateId
_ = ad_feature_df.select("adgroupId", "cateId")
# 由于这里数据集其实很少，所以我们再直接转成Pandas dataframe来处理，把数据载入内存
pdf = _.toPandas()


# 手动释放一些内存
del df
del ad_feature_df
del _
import gc
gc.collect()

34

In [4]:
pdf

,adgroupId,cateId
0,63133,6406
1,313401,6406
2,248909,392
3,208458,392
4,110847,7211
5,607788,6261
6,375706,4520
7,11115,7213
8,24484,7207
9,28589,5953


In [15]:
# 根据指定的类别找到对应的广告
import numpy as np
pdf.where(pdf.cateId==11156).dropna().adgroupId

np.random.choice(pdf.where(pdf.cateId==11156).dropna().adgroupId.astype(np.int64), 200)

313       138953.0
314       467512.0
1661      140008.0
1666      238772.0
1669      237471.0
1670      238761.0
1671        9933.0
4095       53437.0
7540       39952.0
7541      177307.0
7542       85647.0
8692       82370.0
10125     650941.0
11395     189810.0
15450     415875.0
18390     131934.0
18412     422911.0
18415     683715.0
21001      13858.0
23945     102299.0
29187     348378.0
29188     198096.0
30675      85423.0
35256     331234.0
37993      60736.0
38053     749642.0
39503     525090.0
39504     514192.0
43756     334249.0
45583       7306.0
            ...   
825011    447492.0
826573    333855.0
826574    334026.0
826593    339134.0
826594    374667.0
828413    187501.0
830215    729926.0
830319    251428.0
831035    359758.0
831347    269973.0
832404    494080.0
832445    507119.0
832472    481616.0
832473    494730.0
832474    494193.0
832494    481589.0
832719    268994.0
832720    214094.0
832721    236157.0
835127    526529.0
836797    762730.0
837623    48

In [6]:
# 利用ALS模型进行类别的召回

# 加载als模型，注意必须先有spark上下文管理器，即sparkContext，但这里sparkSession创建后，自动创建了sparkContext

from pyspark.ml.recommendation import ALSModel
# 从hdfs加载之前存储的模型
als_model = ALSModel.load("hdfs://hadoop-master:9000/workspace/3.rs_project/project1/trained_result/models/userCateRatingALSModel.obj")
als_model

ALS_4aa2b696592db2d7d3b2

In [7]:
# 返回模型中关于用户的所有属性   df:   id   features
als_model.userFactors

DataFrame[id: int, features: array<float>]

In [8]:
import pandas as pd
cateId_df = pd.DataFrame(np.array(list(set(pdf.cateId))).reshape(6769,1), columns=["cateId"])
cateId_df

,cateId
0,1
1,2
2,3
3,4
4,5
5,6
6,7
7,8
8,9
9,10


In [10]:
cateId_df.insert(0, "userId", np.array([8 for i in range(6769)]))

In [11]:
cateId_df

,userId,cateId
0,8,1
1,8,2
2,8,3
3,8,4
4,8,5
5,8,6
6,8,7
7,8,8
8,8,9
9,8,10


In [14]:
# 传入 userid、cataId的df，对应预测值进行排序
als_model.transform(spark.createDataFrame(cateId_df)).sort("prediction", ascending=False).na.drop().show()

+------+------+----------+
|userId|cateId|prediction|
+------+------+----------+
|     8|  7214|  9.917084|
|     8|   877|  7.479664|
|     8|  7266| 7.4762917|
|     8| 10856| 7.3395424|
|     8|  4766|  7.149538|
|     8|  7282| 6.6835284|
|     8|  7270| 6.2145095|
|     8|   201| 6.0623236|
|     8|  4267| 5.9155636|
|     8|  7267|  5.838009|
|     8|  5392| 5.6882005|
|     8|  6261| 5.6804466|
|     8|  6306| 5.2992325|
|     8| 11050|  5.245261|
|     8|  8655| 5.1701374|
|     8|  4610|  5.139578|
|     8|   932|   5.12694|
|     8| 12276| 5.0776596|
|     8|  8071|  4.979195|
|     8|  6580| 4.8523283|
+------+------+----------+
only showing top 20 rows



In [7]:
import numpy as np
import pandas as pd

import redis

# 存储用户召回，使用redis第9号数据库，类型：sets类型
client = redis.StrictRedis(host="192.168.199.88", port=6379, db=9)

for r in als_model.userFactors.select("id").collect():
    
    userId = r.id
    
    cateId_df = pd.DataFrame(np.array(list(set(pdf.cateId))).reshape(6769,1), columns=["cateId"])
    cateId_df.insert(0, "userId", np.array([userId for i in range(6769)]))
    ret = set()
    
    # 利用模型，传入datasets(userId, cateId)，这里控制了userId一样，所以相当于是在求某用户对所有分类的兴趣程度
    cateId_list = als_model.transform(spark.createDataFrame(cateId_df)).sort("prediction", ascending=False).na.drop()
    # 从前20个分类中选出500个进行召回
    for i in cateId_list.head(20):
        need = 500 - len(ret)    # 如果不足500个，那么随机选出need个广告
        ret = ret.union(np.random.choice(pdf.where(pdf.cateId==i.cateId).adgroupId.dropna().astype(np.int64), need))
        if len(ret) >= 500:    # 如果达到500个则退出
            break
    client.sadd(userId, *ret)
    
# 如果redis所在机器，内存不足，会抛出异常

ResponseError: MISCONF Redis is configured to save RDB snapshots, but is currently not able to persist on disk. Commands that may modify the data set are disabled. Please check Redis logs for details about the error.